In [44]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm

# Models
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss, accuracy_score, f1_score

# PyTorch
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import TensorDataset, DataLoader

# Data Loading 
os.chdir("..")
from data.loader import load_training, load_validation, load_testing, n_freq, n_time, n_classes

### Loading data

In [15]:
X_train, y_train = load_training()
X_val, y_val = load_validation()
X_test, y_test = load_testing()

X_train.shape, X_val.shape, X_test.shape

Use this function only for the very final evaluation before the competition ends.


((33849, 32, 96), (8463, 32, 96), (10578, 32, 96))

### Dummy model (baseline): Predicting the most frequent class

In [42]:
clf_dummy = DummyClassifier(strategy="most_frequent")
clf_dummy.fit(X_train, y_train)

# Predict on Validation Set and Testing set
y_pred_val = clf_dummy.predict(X_val)
y_pred_test = clf_dummy.predict(X_test)

print(f"Number of parameters: \t {len(clf_dummy.get_params())}")
print()
print(f"Testing accuracy: \t{accuracy_score(y_test, y_pred_test)}, \tF1: {f1_score(y_test, y_pred_test, average = 'macro')}, \tlog loss: {log_loss(y_test, clf_dummy.predict_proba(X_test))}")

Number of parameters: 	 3

Testing accuracy: 	0.5169219134051806, 	F1: 0.1363081141717562, 	log loss: 17.41189911310159


### Logistic Regression

In [4]:
clf_lreg = LogisticRegression(
    multi_class="multinomial",
)
clf_lreg.fit(X_train.reshape(-1, n_freq * n_time), y_train)

# Predict on testing set
y_pred_test = clf_lreg.predict(X_test.reshape(-1, n_freq * n_time))

print(f"Number of parameters: \t {32 * 96 * 5 + 1}")
print()
print(f"Testing accuracy: \t{accuracy_score(y_test, y_pred_test)}, \tF1: {f1_score(y_test, y_pred_test, average = 'macro')}, \tlog loss: {log_loss(y_test, clf_lreg.predict_proba(X_test.reshape(-1, n_freq * n_time)))}")

c:\Users\rebek\miniconda3\envs\oticon\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Number of parameters: 	 6

Validation accuracy: 	0.6256646579227224, 	F1: 0.3352538981891634, 	log loss: 0.9949750429151191
Testing accuracy: 	0.6179807146908678, 	F1: 0.3196391238192207, 	log loss: 0.9981086430667209


### MLP (Multi-layer Perceptron): Simple Neural Network

In [9]:
# Lets use a simple neural network as a baseline
clf_mlp = MLPClassifier(
    hidden_layer_sizes=(128, 128),
    max_iter=1000,
    early_stopping=True,
    validation_fraction=0.2,
    random_state=133742069,
)

clf_mlp.fit(X_train.reshape(-1, n_freq * n_time), y_train)

y_pred_test = clf_mlp.predict(X_test.reshape(-1, n_freq * n_time))

print(f"Number of parameters: \t {sum([a.size for a in clf_mlp.coefs_]) +  sum([a.size for a in clf_mlp.intercepts_])}")
print()
print(f"Testing accuracy: \t{accuracy_score(y_test, y_pred_test)}, \tF1: {f1_score(y_test, y_pred_test, average = 'macro')}, \tlog loss: {log_loss(y_test, clf_mlp.predict_proba(X_test.reshape(-1, n_freq * n_time)))}")

Number of parameters: 	 410501

Testing accuracy: 	0.6929476271506901, 	F1: 0.47741689749917027, 	log loss: 0.8317131750195447


### LeNet-alike architecture

In [10]:
force_cpu = False

if t.cuda.is_available() and not force_cpu:
    device = t.device("cuda")
else:
    device = t.device("cpu")

In [16]:
# Convert to tensors
X_map = lambda X: t.from_numpy(X).to(dtype=t.float)
y_map = lambda y: t.from_numpy(y).to(dtype=t.uint8)
loader_map = lambda data: DataLoader(
    dataset=data,
    batch_size=128,
    shuffle=True,
    num_workers=4,
    pin_memory="cuda" == device,
)

X_train, X_val, X_test = map(X_map, (X_train, X_val, X_test))
y_train, y_val, y_test = map(y_map, (y_train, y_val, y_test))

data_train = TensorDataset(X_train, y_train)
data_val = TensorDataset(X_val, y_val)
data_test = TensorDataset(X_test, y_test)

loader_train, loader_val, loader_test = map(loader_map, (data_train, data_val, data_test))

n_train = len(X_train)
n_val = len(X_val)
n_test = len(X_test)

In [12]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            nn.Conv1d(32, 32, 3),
            nn.Conv1d(32, 16, 3),
            nn.Tanh(),
            nn.AvgPool1d(3),
            nn.Conv1d(16, 4, 1),
            nn.Tanh(),
            nn.Flatten(),
            nn.Linear(120, 64),
            nn.Tanh(),
            nn.Linear(64, n_classes)
        )

    def forward(self, X: t.Tensor) -> t.Tensor:
        return self.model(X)

learning_rate = 1e-4
weight_decay = 1e-4
n_epochs = 20

model = Model().to(device)
print(f"Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")


model.train()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

acc_running = 0

for epoch in (bar:=trange(n_epochs)):
    for x, y in loader_train:
        x = x.to(device)
        y = y.to(device)

        y_pred = model(x)
        
        loss = loss_fn(y_pred, y)

        acc = (y_pred.argmax(dim=1) == y).sum() / y.size(0)
        acc_running += 0.05 * (acc.item() - acc_running)
        bar.set_postfix(acc=f"{acc_running:.2f}")

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Parameters: 12793


  0%|          | 0/20 [00:00<?, ?it/s]

In [32]:
model.eval()

y_pred = t.zeros(n_test, n_classes, device=device)

for i, (x, y) in enumerate(loader_test):
    x = x.to(device)
    y = y.to(device)
    
    y_pred[i*128: 128 + i*128,:] = model(x)

print(f"Number of parameters: \t {sum(p.numel() for p in model.parameters())}")
print()
print(f"Testing accuracy: \t{accuracy_score(y_test, y_pred.argmax(dim=1).detach().numpy())}, \tF1: {f1_score(y_test, y_pred.argmax(dim=1).detach().numpy(), average = 'macro')}, \tlog loss: {log_loss(y_test, y_pred.detach().numpy())}")

Number of parameters: 	 12793

Testing accuracy: 	0.4155795046322556, 	F1: 0.18802420876187903, 	log loss: 6.588488782856262
